# NB:
- not using cooccurrences at all
- computing only equivalent sources by simple rules

In [ ]:
PROJECT_NAME='Climate change'
# PROJECT_NAME='COVID-19'
# PROJECT_NAME='Biology_sample'
# PROJECT_NAME='History_sample'
# PROJECT_NAME='Media_sample'

LANGUAGE='english'
# LANGUAGE='other_languages'

In [ ]:
import pandas as pd
from copy import deepcopy
import numpy as np

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from utils_domain import  get_url_domain

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from paths import get_path_urlsinfo, get_path_collected

In [ ]:
PATHNAME_INPUT=f"{get_path_collected(PROJECT_NAME, LANGUAGE)}sources_revision_edits.csv"

In [ ]:
PATHNAME_INPUT_URLSINFO=get_path_urlsinfo()+'urls_info_and_domain.csv'

In [ ]:
PATH_OUTPUT=get_path_collected(PROJECT_NAME, LANGUAGE)

### Source cooccurrences

In [ ]:
SOURCES_REV=pd.read_csv(PATHNAME_INPUT)

In [ ]:
len(SOURCES_REV)

### URLS Info (redirect+domain)

In [ ]:
URLS_INFO=pd.read_csv(PATHNAME_INPUT_URLSINFO)

In [ ]:
# URLS2DOMAIN  =URLS_INFO.set_index('url')['destination_domain'].to_dict() #not used atm

In [ ]:
URLS2REDIRECT=URLS_INFO.set_index('url')['destination_url'].to_dict()
# URLS2REDIRECT=URLS_INFO[URLS_INFO.status_code=='200'].set_index('url')['destination_url'].to_dict()

In [ ]:
URLS_INFO

In [ ]:
SOURCES_REV['ts']=SOURCES_REV['source_type']+'|$|'+SOURCES_REV['source']
TS_EDIT_DATA=SOURCES_REV['ts'].unique()

In [ ]:
len(TS_EDIT_DATA)

# Brief analysis (before re-elaborating it)

Counting presence by source type

In [ ]:
PATHNAME_INPUT

In [ ]:
assert len(SOURCES_REV[SOURCES_REV.source_type.isna()])==0

In [ ]:
SOURCES_REV.source_type.map(lambda x: x.split('|$|')[0]).value_counts(normalize=True)*100

# Computing equivalence groups

In [ ]:
def _update_equivalence_stats(expansion_type, expansion_triplet):
    global EQUIVALENCE_STATS
    
    if expansion_type not in EQUIVALENCE_STATS: EQUIVALENCE_STATS[expansion_type]=set()
    EQUIVALENCE_STATS[expansion_type].update([tuple(expansion_triplet)])

def _update_equivalence_count(t, relatives):
    global EQUIVALENCE_COUNT
    
    if t not in EQUIVALENCE_COUNT: EQUIVALENCE_COUNT[t]={}
    n_rel=len(relatives)-1
    if n_rel not in EQUIVALENCE_COUNT[t]: EQUIVALENCE_COUNT[t][n_rel]=0
    EQUIVALENCE_COUNT[t][n_rel]+=1 

NB check that this function works as it should

In [ ]:
def strip_section(ts):
    t,s =ts.split('|$|')
    if not t in ['unstructurl', 'url', 'archive', 'chapter', 'trunc-arch','trunc-sect']: return ts
    loc_sect=s.rfind('#')
    loc_page=s.rfind('&page=')
    loc_pg=s.rfind('&pg=')
    loc_qm=s.find('?')

    min_loc=[loc for loc in [loc_sect, loc_page, loc_pg] if loc>0 ] #-1 does not count
    min_loc=sorted(min_loc)
    if len(min_loc): 
        return 'trunc-sect|$|'+s[:min_loc[0]]
    else:
        return ts
    
strip_section('chapter|$|https://books.google.com/books?id=UuAIyYdrHJQC&pg=PA181#v=onepage') #example

In [ ]:
def get_redirect_identifier(ts, resilient=False):
    t,s=ts.split('|$|')
    if t not in ['unstructurl', 'url', 'archive', 'chapter', 'trunc-arch','trunc-sect']: return  ts
    if 'trunc' in t: resilient=True
    # resilient NEEDS to be true when using url from truncated urls
#     else:              resilient=False
    redirect_relative=URLS2REDIRECT.get(s, None)
    if redirect_relative==None: 
        if not resilient:
            print(f"NOT FOUND 1 {t}|$|{s} IN URLS_INFO: values found")
            assert 0
        return ts
    if redirect_relative=='FAILED' or redirect_relative==s: return ts
    return 'url|$|'+redirect_relative

In [ ]:
import networkx as nx

In [ ]:
def _update_equivalence_groups(l_ts):
    global EQUIVALENCE_GRAPH
    global GROUPED_TS
    
    if len(l_ts)==0: return
    if len(l_ts)==1: EQUIVALENCE_GRAPH.add_node(list(l_ts)[0])
    
    if l_ts & GROUPED_TS: #it means that at least one of them has been grouped
        if l_ts - GROUPED_TS: # it means at least one of them has not been grouped 
            for i, ts_i in enumerate(l_ts):
                for j, ts_j in enumerate(l_ts):
                    if i<j: continue
                    EQUIVALENCE_GRAPH.add_edge(ts_i, ts_j)
            GROUPED_TS.update(l_ts)
        else: # it means that all of them are already grouped
            return
    else:
        for i, ts_i in enumerate(l_ts):
            for j, ts_j in enumerate(l_ts):
                if i<j: continue
                EQUIVALENCE_GRAPH.add_edge(ts_i, ts_j)
        GROUPED_TS.update(l_ts)

In [ ]:
def compute_relative_identifiers(initial_ts):
    initial_t, s=initial_ts.split('|$|')
    
    if initial_t not in ['doi', 'unstructurl', 'url', 'archive', 'chapter']: return 
    
    relative_identifiers={initial_ts:f'{initial_t:-<12}->>>-{"self":->10}'}
    
    if initial_t=='doi':
        _t_exp_type=f'{"doi":-<3}->>>-{"url":->3}'
        new_ts='url|$|https://doi.org/'+s
        relative_identifiers[new_ts]=_t_exp_type
        _update_equivalence_stats(_t_exp_type, [initial_ts, new_ts])
    
        # expand by redirect #ONLY FOR DOI URLS
        for r_ts, r_type in deepcopy(relative_identifiers).items():
            _t_exp_type = '{:-<{}}->>>-{:->10}'.format(r_type, 12, 'redirect') 
            new_ts =get_redirect_identifier(r_ts, resilient=False)  #if fails, returns r_ts itself
            if new_ts not in relative_identifiers:
                relative_identifiers[new_ts]=_t_exp_type
                _update_equivalence_stats(_t_exp_type, [initial_ts, r_ts, new_ts])
    
#     if t in ['unstructurl', 'url', 'archive', 'chapter']:
    _t_exp_type=f'{initial_t:-<12}->>>-{"trunc-arch":->10}'
    if get_url_domain(s)=='web.archive.org':
        if s.rfind('http')>0:
            new_ts='trunc-arch|$|'+s[s.rfind('http'):]
            relative_identifiers[new_ts]=_t_exp_type
            _update_equivalence_stats(_t_exp_type, [initial_ts, new_ts])

    # expand by removing pg,page and sections in urls
    for r_ts, r_type in deepcopy(relative_identifiers).items():
        _t_exp_type = '{:-<{}}->>>-{:->10}'.format(r_type, 12+5+10, 'trunc-sect')
        new_ts=strip_section(r_ts)     #if fails, returns r_ts itself
        if new_ts not in relative_identifiers:
            relative_identifiers[new_ts]=_t_exp_type
            _update_equivalence_stats(_t_exp_type, [initial_ts, r_ts, new_ts])

            
#  EQUIVALENCE BY REDIRECT IS DANGEROUS
#  BECAUSE urls can redirect to very generic urls (ex web domain itself or web domain + error page)
#  so there is risk of collapsing big portions of the data to a generic url
#     # expand by redirect
#     for r_ts, r_type in deepcopy(relative_identifiers).items():
#         _t_exp_type = '{:-<{}}->>>-{:->10}'.format(r_type, 12+5+10+5+10, 'redirect') 
#         new_ts =get_redirect_identifier(r_ts)  #if fails, returns r_ts itself
#         if new_ts not in relative_identifiers:
#             relative_identifiers[new_ts]=_t_exp_type
#             _update_expansion_stats(_t_exp_type, [ts, r_ts, new_ts])

    # COLLAPSING all url types to url #
    # NB not including for doi code BUT including https:... doi.org/{doi_code})
    set_clean_ts=[]
    set_clean_ts_without_initial=[]
    for ts in relative_identifiers.keys():
        t,s=ts.split('|$|')
        if t!='doi': clean_ts='url|$|'+s
        if t=='doi': clean_ts=ts
        set_clean_ts.append(clean_ts)
#         if ts!=initial_ts:
#             set_clean_ts_without_initial.append(clean_ts)
    set_clean_ts=set(set_clean_ts)
#     set_clean_ts_without_initial=set(set_clean_ts_without_initial)
    
    
    _update_equivalence_groups(set_clean_ts)   
    if initial_ts not in _counted_equiv:
        _update_equivalence_count(initial_t, set_clean_ts)
        _counted_equiv.update([initial_ts])


In [ ]:
# EXPANSION_DATA=[]

# for _, row in tqdm(SOURCES_COOCC.iterrows(), total=len(SOURCES_COOCC)):
#     ts_i=row.source_id_i
#     ts_j=row.source_id_j
    
#     relative_id_i=get_relative_identifiers(ts_i)
#     relative_id_j=get_relative_identifiers(ts_j)

#     TO_ADD=[]
#     for rid_i in relative_id_i:
#         row_exp=deepcopy(row)
#         row_exp.source_id_i=rid_i
#         TO_ADD.append(row_exp)
        
#     for rid_j in relative_id_j:
#         row_exp=deepcopy(row)
#         row_exp.source_id_j=rid_j
#         TO_ADD.append(row_exp)
        
#     for rid_i in relative_id_i:
#         for rid_j in relative_id_j:
#             row_exp=deepcopy(row)
#             row_exp.source_id_i=rid_i
#             row_exp.source_id_j=rid_j
#             TO_ADD.append(row_exp)    
            
#     EXPANSION_DATA+=TO_ADD

In [ ]:
EQUIVALENCE_GRAPH=nx.Graph()
GROUPED_TS=set()
_counted_equiv=set()
EQUIVALENCE_STATS={}
EQUIVALENCE_COUNT={}

In [ ]:
for n_ts, ts in enumerate(tqdm(SOURCES_REV['ts'], total=len(SOURCES_REV['ts']))):
    compute_relative_identifiers(ts)


### Extract equivalent groups
(they are stored in a graph as connected components)

In [ ]:
EQUIVALENCE_GROUPS=[]
for eg in tqdm(nx.connected_components(EQUIVALENCE_GRAPH)):
    EQUIVALENCE_GROUPS.append(eg)          

# Statistics of the expansion

NB the expansion can start from any of doi+all the url variations

and map them to ALWAYS a type=url  

In [ ]:
for t, ex_c in EQUIVALENCE_COUNT.items():
    for size, occurrences in ex_c.items():
        print(f"{t:12} mapped to  {size:2} urls;  occurrences: {occurrences}")
    print()

In [ ]:
srt_exp_stats={t: data for t, data in sorted(EQUIVALENCE_STATS.items(), key=lambda x:x[0])}
for exp_type, exp_pairs in srt_exp_stats.items():
    print(f"{exp_type: <60}:", len(exp_pairs))

In [ ]:
# EQUIVALENCE_STATS['doi->>>-url']

In [ ]:
# '10.1637/0003-0031-185.1.32' in SOURCES_REV['source'].values
# 'https://doi.org/10.1637/0003-0031-185.1.32' in URLS_INFO['url'].values
# URLS_INFO[URLS_INFO['url']=='https://doi.org/10.1637/0003-0031-185.1.32']
# len(set([d[0] for d in EQUIVALENCE_STATS['doi->>>--url']])-set([d[0] for d in EQUIVALENCE_STATS['doi->>>--url->>>---redirect']]))
# set([d[0] for d in EQUIVALENCE_STATS['doi->>>--url']])-set([d[0] for d in EQUIVALENCE_STATS['doi->>>--url->>>---redirect']])

Checking no double occurrences in groups

In [ ]:
_grouped_ts=set()

for eg in tqdm(EQUIVALENCE_GROUPS):
    assert len(_grouped_ts&eg)==0  
    _grouped_ts.update(eg)


In [ ]:
len(_grouped_ts)

Check ts are of correct typing

In [ ]:
for ts in _grouped_ts:
    assert ts.split('|$|')[0] in ['doi', 'url']

Checking every original s is currently in storage

In [ ]:
_grouped_s=set([ts.split('|$|')[-1] for ts in _grouped_ts])

for t, s in zip(SOURCES_REV['source_type'], SOURCES_REV['source']):
    if t not in ['doi', 'unstructurl', 'url', 'archive', 'chapter']: continue
                
    try:
        s in _grouped_s
    except:
        print(s)
        assert 0
print("everything fine")

#### Statistics of equivalence groups

In [ ]:
SIZE2EGS={}
for eg in EQUIVALENCE_GROUPS:
    size=len(eg)
    if size not in SIZE2EGS:
        SIZE2EGS[size]=[]
    SIZE2EGS[size].append(eg)

SIZE2EGS={s:egs for s, egs in sorted(SIZE2EGS.items(), key=lambda x: x[0])}
    
for size, egs in SIZE2EGS.items():
    print(f"groups of size {size:2}: occurrences {len(egs)}")

In [ ]:
# SIZE2EGS[7]

# Choosing best identifier for each group

In [ ]:
S_OCCURRENCE=\
SOURCES_REV[SOURCES_REV['source_type'].isin(['doi', 'url', 'archive', 'unstructurl', 'chapter'])]\
['source'].value_counts().to_dict()

In [ ]:
len(S_OCCURRENCE)

In [ ]:
TS_TO_EQUIVALENT_URL={}
for eg in EQUIVALENCE_GROUPS:
    
    # ------------------------------- first chooses the best ts
    best_t='url'
    best_ts=''
    best_count=0
    for this_ts in eg:
        this_t, this_s = this_ts.split('|$|')
        this_count=S_OCCURRENCE.get(this_s,0)
        
        if this_t=='doi': 
            best_t    =this_t
            best_ts   ='url|$|https://doi.org/'+this_s
            best_count=this_count    
            
        if 'web.archive.org' in this_s:
            this_t='archive' 
            
            best_t    =this_t
            best_ts   =this_ts
            best_count=this_count
            
        if best_t=='archive' and this_t in ['url']: 
            continue
            
        if best_t=='doi' and this_t in ['url', 'archive']: 
            continue
        
        if this_count>=best_count:
            best_count=this_count
            if this_t=='doi':
                best_ts   ='url|$|https://doi.org/'+this_s
            else:
                best_ts   =this_ts
    
    for ts in eg:
        assert ts not in TS_TO_EQUIVALENT_URL
        TS_TO_EQUIVALENT_URL[ts]=best_ts

In [ ]:
len(TS_TO_EQUIVALENT_URL)

In [ ]:
for ts_i, ts_j in TS_TO_EQUIVALENT_URL.items():
    t_i, s_i =ts_i.split('|$|')
    t_j, s_j =ts_j.split('|$|')
    
    try:
        assert t_j=='url'
    except:
        print(t_j)
        break
    try:
        assert t_i in ['url','doi']
    except:
        print(t_i)
        break

# Saving

In [ ]:
import pickle

In [ ]:
# TS_TO_EQUIVALENT_TS

In [ ]:
PATH_OUTPUT

In [ ]:
with open(f'{PATH_OUTPUT}ts_to_equivalent.pickle', 'wb') as handle:
    pickle.dump(TS_TO_EQUIVALENT_URL, handle)

In [ ]:
with open(f'{PATH_OUTPUT}equivalence_groups.pickle', 'wb') as handle:
    pickle.dump(EQUIVALENCE_GROUPS, handle)

In [ ]:
fermoqui
codice vecchio

In [ ]:
with open(f'{PATH_OUTPUT}ts_to_equivalent.pickle', 'rb') as handle:
    tst=pickle.load(handle)

In [ ]:
len(tst)==len(TS_TO_EQUIVALENT_TS)